In [1]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}
// Note that this must be in its own cell

<IPython.core.display.Javascript object>

In [2]:
# External imports
import ipywidgets as ipw
#from jinja2 import Environment
#from importlib_resources import files

from aiida.orm import StructureData
from aiida.orm import load_node
from aiidalab_widgets_base import StructureBrowserWidget, StructureManagerWidget, StructureUploadWidget
from aiidalab_widgets_base import SmilesWidget
from aiidalab_widgets_base import BasicStructureEditor
from aiidalab_widgets_base import WizardAppWidget
#from aiidalab_widgets_base.bug_report import install_create_github_issue_exception_handler

In [3]:
#from aiidalab_ispg.process import WorkChainSelector
from aiidalab_ispg.steps import SubmitQeAppWorkChainStep
from aiidalab_ispg.structures import StructureSelectionStep
from aiidalab_ispg.steps import ViewQeAppWorkChainStatusAndResultsStep
from aiidalab_ispg.steps import ViewSpectrumStep

# DH: Our own hacked molecule viewer
from aiidalab_ispg.widget_viewers.viewers import StructureDataViewer

# DH This is for later for importing Jinja templates and styles
#from aiidalab_ispg import static

structure_manager_widget = StructureManagerWidget(
    importers=[
        StructureUploadWidget(title="Upload file"),
        SmilesWidget(title='SMILES'),
        StructureBrowserWidget(title="AiiDA database"),
    ],
    node_class='StructureData',
    viewer=StructureDataViewer(downloadable=False,storable=False),
#    editors = [
#        BasicStructureEditor(title="Basic Editor"),
#    ],
)

structure_selection_step = StructureSelectionStep(manager=structure_manager_widget)
structure_selection_step.auto_advance = True

submit_orca_work_chain_step = SubmitQeAppWorkChainStep()
submit_orca_work_chain_step.auto_advance = True

view_orca_work_chain_status_and_results_step = ViewQeAppWorkChainStatusAndResultsStep()
view_orca_work_chain_status_and_results_step.auto_advance = True

view_spectrum_step = ViewSpectrumStep()

# Link the application steps
ipw.dlink((structure_selection_step, 'confirmed_structure'), (submit_orca_work_chain_step, 'input_structure'))
ipw.dlink((submit_orca_work_chain_step, 'process'), (view_orca_work_chain_status_and_results_step, 'process'))
ipw.dlink((view_orca_work_chain_status_and_results_step, 'process'), (view_spectrum_step, 'process'))

# Add the application steps to the application
app = WizardAppWidget(
    steps=[
        ('Select structure', structure_selection_step),
        ('Submit work chain', submit_orca_work_chain_step),
        ('Status & Results', view_orca_work_chain_status_and_results_step),
        ('Spectrum', view_spectrum_step),
    ])

# Add process selection header
#work_chain_selector = WorkChainSelector(layout=ipw.Layout(width='auto'))

# Reset all subsequent steps in case that a new structure is selected
def _observe_structure_selection(change):
    with structure_selection_step.hold_sync():
        if structure_selection_step.confirmed_structure is not None and \
                structure_selection_step.confirmed_structure != change['new']:
            app.reset()
structure_selection_step.observe(_observe_structure_selection, 'structure')

display(app)

WizardAppWidget(children=(HBox(children=(Button(description='Previous step', disabled=True, icon='step-backwar…

In [95]:
from bokeh.plotting import figure
from bokeh.io import show, output_notebook
output_notebook()
p = figure(title="Spectrum test",  x_axis_label='E / eV',   y_axis_label='I / cm^-2 * molecule ^ -1')
p.line(x, y, legend_label='legend title', line_width=2)
p.line(x, y, line_width=2)
show(p)

Loading BokehJS ...

In [26]:
#for xi, yi in zip(x, y):
#    print(xi, yi)

In [57]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [64]:
import bqplot.pyplot as plt
import ipywidgets as widgets

def plot_spectrum(width, kernel):
    transition1 = {
        'energy': 1, # Excited energy in eV
        'osc_strength': '0.016510951'
    }
    transition2 = {
        'energy': 2.0, # Excited energy in eV
        'osc_strength': '0.05'
    }
    transitions = [transition1, transition2]
    nsample = 1
    spec = Spectrum(transitions, nsample)
    if kernel == 'lorentzian':
        x, y = spec.get_lorentzian_spectrum(width, 'ev', 'cross_section')
    elif kernel == 'gaussian':
        x, y = spec.get_gaussian_spectrum(width, 'ev', 'cross_section')
    else:
        print("ERROR")

    fig = plt.figure(title="title title")  
    plt.plot(x, y)
    plt.xlabel("Energy")
    plt.ylabel("Intensity")
    plt.show()

In [65]:
_ = widgets.interact(plot_spectrum, width=(0.05, 1.0, 0.05), kernel=['gaussian', 'lorentzian'])

interactive(children=(FloatSlider(value=0.5, description='width', max=1.0, min=0.05, step=0.05), Dropdown(desc…

In [11]:
%aiida
n = load_node(pk=255)
output_params = n.outputs.output_parameters.get_dict()

In [12]:
output_params

dict_keys(['nmo', 'mult', 'homos', 'natom', 'charge', 'etoscs', 'etsecs', 'etsyms', 'nbasis', 'atomnos', 'moments', 'elements', 'etrotats', 'metadata', 'scfvalues', 'transprop', 'atomcoords', 'atommasses', 'etenergies', 'moenergies', 'scftargets', 'atomcharges', 'scfenergies', 'coreelectrons'])

In [16]:
en = output_params['etenergies']
osc = output_params['etoscs']

In [27]:
CM2EV = 1 / 8065.7
transitions = [{'energy': tr[0]*CM2EV, 'osc_strength': tr[1] } for tr in zip(en, osc)]

In [28]:
transitions

[{'energy': 4.416082919027487, 'osc_strength': 0.0},
 {'energy': 4.617131805051018, 'osc_strength': 0.0},
 {'energy': 4.618024474007216, 'osc_strength': 0.0}]

In [197]:
import traitlets
import matplotlib.pyplot as plt

class SpectrumWidget(ipw.VBox):

    transitions = traitlets.List()
    
    def __init__(self, **kwargs):
        title = ipw.HTML(
            """<div style="padding-top: 0px; padding-bottom: 0px">
            <h4>UV/Vis Spectrum</h4></div>"""
        )
        self.output = ipw.Output()
        self.spectrum_container = ipw.Box()
        self.width_slider = ipw.FloatSlider(min=0.05,max=1,step=0.05, value=0.5, description="Width / eV")
        self.kernel_selector = ipw.ToggleButtons(
            options=['gaussian', 'lorentzian'], # TODO: None option
            description='Broadening kernel:',
                disabled=False,
                button_style='info', # 'success', 'info', 'warning', 'danger' or ''
                tooltips=['Description of slow', 'Description of regular', 'Description of fast'],
        )
        
        super().__init__([title, self.kernel_selector, self.width_slider, self.spectrum_container, self.output], **kwargs)
        
    def _plot_spectrum(self, kernel, width):
        # TODO: Pass in number of samples somehow
        
        if not self._validate_transitions():
            return
        
        nsample = 1
        spec = Spectrum(transitions, nsample)
        energy_unit = 'eV'
        intensity_unit = 'cm^-1'
        if kernel == 'lorentzian':
            x, y = spec.get_lorentzian_spectrum(width, energy_unit, intensity_unit)
        elif kernel == 'gaussian':
            x, y = spec.get_gaussian_spectrum(width, energy_unit, intensity_unit)
        else:
            with self.output:
                print("Invalid broadening type")
                return

        # Determine min max of x and y axes so that they
        # don't change when changing width
        fig = plt.figure()  
        plt.plot(x, y)
        plt.xlabel(f"Energy / {energy_unit}")
        plt.ylabel(f"Intensity / {intensity_unit}")
        plt.show()
    
    def _validate_transitions(self):
        # TODO: Validate that transitions list is correct
        for tr in self.transitions:
            if not isinstance(tr, dict) or ( 
                'energy' not in tr or
                'osc_strength' not in tr):
                with self.output:
                    print('Invalid transition', tr)
                    return False
                    
        return True
    
    def _show_spectrum(self):
        self.output.clear_output()
        if self._validate_transitions:
            spectrum = ipw.interactive_output(
                self._plot_spectrum,
                {'width': self.width_slider, 
                 'kernel': self.kernel_selector}
            )
            self.spectrum_container.children = [spectrum]
    
    @traitlets.observe("transitions")
    def _observe_transitions(self, change):
        self._show_spectrum()

In [198]:
w = SpectrumWidget()
#s = w._plot_spectrum('gaussian', 0.05)
w

SpectrumWidget(children=(HTML(value='<div style="padding-top: 0px; padding-bottom: 0px">\n            <h4>UV/V…

In [199]:
w.transitions = transitions
#b = ipw.Box()
#b.children

In [5]:
max(2,1)

2

In [38]:
from traitlets import TraitType
# TODO: Define our own TraitType for electronic transitions
# with custom validation logic
class ElectronicTransition(TraitType):
    """List of electronic transitions in dict type in atomic units"""
    
    info_text = 'an (energy, osc_strength) tuple'
    
    def validate(self, obj, value):
        if isinstance(value, dict):
            if 'energy' in value and 'osc_strength' in value and len(dict.keys()) == 2:
                return value
        self.error(obj, value)
        
e = ElectronicTransition()